In [19]:
import pandas as pd
import numpy as np
base = "../../data/"
target = pd.read_csv(base+"processed/cleaned_target.csv")
target.columns

Index(['Year', 'Type', 'LGA', 'Assault - domestic violence',
       'Assault - non-domestic violence', 'Sexual Offences', 'Robbery',
       'Break and enter dwelling', 'Break and enter non-dwelling',
       'Motor vehicle theft', 'Steal from motor vehicle',
       'Steal from retail store', 'Steal from dwelling', 'Steal from person',
       'Fraud', 'Malicious damage to property', 'Harassment threatening',
       'Receiving stolen goods', 'Other theft', 'Arson',
       'Possession use of cannabis', 'Prohibited weapons offences', 'Trespass',
       'Offensive conduct', 'Offensive language', 'Liquor offences',
       'Breach AVO', 'Breach bail conditions', 'Resist or hinder officer',
       'Transport regulatory offences'],
      dtype='object')

In [4]:
target["Type"].unique()

array(['Total', 'Rate per 100,000 population', 'Rank'], dtype=object)

In [20]:
nswdf_target = target[target["Type"]=='Rate per 100,000 population'].dropna()

In [8]:
opal_df = pd.read_csv(base+"processed/opal_lgas.csv")
opal_lgas = opal_df["lga"].unique()

/Users/tdm/git/capstone/venv/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [14]:
set(opal_lgas).difference(set(nswdf_target["LGA"].unique()))

set()

In [21]:
# log it
start_col = nswdf_target.columns.get_loc('Assault - domestic violence')
for i in range(27):
    nswdf_target = nswdf_target.replace(0.0, 0.01)
    nswdf_target.iloc[:,start_col+i] = np.log(nswdf_target.iloc[:,start_col+i])
    
#print(nswdf_target["pop_class"])
#nswdf_target["pop_class"].hist()

nswdf_target = nswdf_target[(nswdf_target["LGA"].isin(opal_lgas)) & (nswdf_target["Year"] == 2016)]

In [22]:
nswdf_target.columns

Index(['Year', 'Type', 'LGA', 'Assault - domestic violence',
       'Assault - non-domestic violence', 'Sexual Offences', 'Robbery',
       'Break and enter dwelling', 'Break and enter non-dwelling',
       'Motor vehicle theft', 'Steal from motor vehicle',
       'Steal from retail store', 'Steal from dwelling', 'Steal from person',
       'Fraud', 'Malicious damage to property', 'Harassment threatening',
       'Receiving stolen goods', 'Other theft', 'Arson',
       'Possession use of cannabis', 'Prohibited weapons offences', 'Trespass',
       'Offensive conduct', 'Offensive language', 'Liquor offences',
       'Breach AVO', 'Breach bail conditions', 'Resist or hinder officer',
       'Transport regulatory offences'],
      dtype='object')

In [28]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from scipy.stats import probplot

from matplotlib import pyplot as plt

def kfold_cv(k, crimenum, dataframe, target):
    lga_names = target["LGA"].unique()
    start_col = target.columns.get_loc('Assault - domestic violence')
    kf = KFold(n_splits=5, shuffle=True)
    
    output = pd.DataFrame(index=range(len(dataframe)))
    for train_index, test_index in kf.split(lga_names):
        train_lgas = lga_names[train_index]
        traindf = dataframe.loc[dataframe["lga"].isin(train_lgas)]
        train_target = target.loc[target["LGA"].isin(train_lgas)]
        
        test_lgas = lga_names[test_index]
        testdf = dataframe.loc[dataframe["lga"].isin(test_lgas)]
        test_target = target.loc[target["LGA"].isin(test_lgas)]
        #import pdb; pdb.set_trace()
        model = RandomForestRegressor()
        model.fit(traindf, train_target.iloc[:,start_col+crimenum])
        
        #print(lga_loc)
        preds = model.predict(testdf)

        #is this a fair measure
        rmse.append(np.sqrt(mean_squared_error(preds, test_target.iloc[:,start_col+crimenum])))
        #probplot(testdf.iloc[:,start_col+crimenum] - mu, dist='norm', plot=plt)
        plt.show()

    return output, rmse

output = kfold_cv(5, 0, opal_df, nswdf_target)
output[1]                                     

ValueError: could not convert string to float: 'NSW'

In [33]:
opal_df.groupby(["lga", "tap", "night", "morning", "afternoon", "evening"]).sum()["count"]

lga                tap  night  morning  afternoon  evening
Bathurst Regional  off  0      0        0          1             328
                   on   0      1        0          0             133
                        1      0        0          0             292
Bayside            off  0      0        0          1          202567
                                        1          0          271611
                               1        0          0          192207
                        1      0        0          0           11450
                   on   0      0        0          1           75328
                                        1          0          267055
                               1        0          0          345234
                        1      0        0          0           19599
Blacktown          off  0      0        0          1          184567
                                        1          0          380235
                               1        0   